In [2]:
%pylab inline
import pandas as pd
from os import listdir
from os.path import isfile, join

Populating the interactive namespace from numpy and matplotlib


In [3]:
mypath = 'WY-Wolves/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
games = []
for f in onlyfiles:
    if('vs.' in f):
        idx = f.index('vs.')
        games.append(f[idx + 4:])

games = list(set(games))
games

['YW-Dogs 2017-03-11_09-15.csv',
 'PH-Dogs 2017-03-03_16-00.csv',
 'CI-Vapour 2017-03-29_16-30.csv',
 'HK-Joint 2017-05-20_09-00.csv',
 'BN-Wolves 2017-04-08_14-45.csv',
 'DQ-Cheetahs 2017-05-12_15-45.csv',
 'NO-Mosquitos1 2017-04-08_11-00.csv',
 'SC-Warriors 2017-05-09_16-30.csv',
 'BJ-Tornados 2017-05-21_09-00.csv',
 'YK-Buddha 2017-03-11_12-30.csv',
 'NO-Mosquitos2 2017-04-08_10-00.csv',
 'YK-Buddha 2017-03-11_14-30.csv',
 'NM-Lions 2017-03-17_16-00.csv',
 'JK-Sea Otters 2017-03-31_15-45.csv',
 'CI-Vapour 2017-05-20_12-00.csv',
 'DQ-Cheetahs 2017-03-11_13-40.csv',
 'KC-Gators 2017-03-11_11-30.csv',
 'TJ-Donut1 2017-04-08_13-30.csv',
 'DQ-Cheetahs 2017-02-24_16-15.csv',
 'CI-HZSR 2017-05-21_15-50.csv',
 'NM-Lions 2017-05-20_10-00.csv',
 'YW-Dogs 2017-03-24_16-00.csv',
 'SC-Warriors 2017-04-21_15-00.csv',
 'NM-Lions 2017-05-21_12-20.csv',
 'LW-Kraken 2017-05-20_15-00.csv']

In [7]:
def single_game_single_thrower(receiver, thrower, df):
    #df should be passed in
    data = {'Drops': len(df[df['Receiver error?'] == 1]),
            'Catches': len(df[df['Turnover?'] == 0]),
            'Throwaways': len(df[df['Thrower error?'] == 1]),
            'Attempts': len(df[df['Receiver error?'] == 1]) + len(df[df['Turnover?'] == 0]) + len(df[df['Thrower error?'] == 1]),
            'Goals': len(df[df['Assist?'] == 1])}
    return pd.Series(data)

def single_game_passing_data(receiver, game):
    df = pd.read_csv('WY-Wolves/Passes vs. ' + game)
    df = df[df['Receiver'] == receiver]
    data = pd.DataFrame(columns=['Drops', 'Catches', 'Catch Pct', 'Throwaways', 'Attempts', 'Goals'], index=['Thrower'])
    for thrower, frame in df.groupby('Thrower'):
        data.loc[thrower] = single_game_single_thrower(receiver, thrower, frame)

    return data

def player_receiving_data(receiver):
    # Statto
    data = single_game_passing_data(receiver, games[0])
    for game in games[1:]:
        data = data.add(single_game_passing_data(receiver, game), fill_value=0)

    return data

In [8]:
player_receiving_data('#14 Cecilia Wu').sort_values('Catches',ascending=False)

,Drops,Catches,Catch Pct,Throwaways,Attempts,Goals
#22 Paul Simpson,4,75,NaN,9,88,5
#27 Daniel Zhang,3,52,NaN,6,61,5
#55 Tigger Li,5,28,NaN,6,39,3
#1 Joyce Wang,2,18,NaN,5,25,0
Oscar Yao,0,7,NaN,0,7,0
#19 Ben Wang,0,3,NaN,2,5,1
#28 Tina Zhu,0,3,NaN,0,3,0
#66 Jerry Chen,0,3,NaN,0,3,0
#0 Julie Luo,0,2,NaN,0,2,0
#76 Jack Xia,0,2,NaN,0,2,0


In [10]:
def ulti_receiving_data(receiver):
    ulti_df = pd.read_csv('WY-Wolves/1617SUISWolves-stats.csv')
    ulti_df = ulti_df[ulti_df['Receiver'] == receiver]
    data = pd.DataFrame(columns=['Drops', 'Catches', 'Catch Pct', 'Throwaways', 'Attempts', 'Goals'], index=['Player'])
    for thrower, frame in ulti_df.groupby('Passer'):
        data.loc[thrower] = pd.Series({'Drops': len(frame[frame['Action'] == 'Drop']),
                                       'Catches': len(frame[frame['Action'] == 'Catch']),
                                       'Throwaways': len(frame[frame['Action'] == 'Throwaway']),
                                       'Attempts': len(frame[frame['Action'] == 'Drop']) + len(frame[frame['Action'] == 'Catch']) + len(frame[frame['Action'] == 'Throwaway']),
                                       'Goals': len(frame[frame['Action'] == 'Goal'])})
    return data

In [11]:
ulti_receiving_data('#14 Cecilia Wu').sort_values('Catches', ascending=False)

,Drops,Catches,Catch Pct,Throwaways,Attempts,Goals
#22 Paul Simpson,4.0,61.0,NaN,0.0,65.0,10.0
#1 Joyce Wang,3.0,17.0,NaN,0.0,20.0,0.0
#27 Daniel Zhang,4.0,15.0,NaN,0.0,19.0,1.0
# 69 Justin Chen,2.0,11.0,NaN,0.0,13.0,0.0
#55 Tigger Li,1.0,10.0,NaN,0.0,11.0,0.0
#66 Jerry Chen,0.0,8.0,NaN,0.0,8.0,0.0
#2 Jamie Chen,0.0,5.0,NaN,0.0,5.0,0.0
Oscar Yao,0.0,3.0,NaN,0.0,3.0,0.0
#39 David Xiao,0.0,3.0,NaN,0.0,3.0,0.0
Chris Xu,0.0,3.0,NaN,0.0,3.0,0.0


In [14]:
receiver = '#22 Paul Simpson'
sdf = player_receiving_data(receiver)
udf = ulti_receiving_data(receiver)
pdf = sdf.add(udf, fill_value=0)

def catch_percent(row):
        catch_pct = 0
        if row['Drops'] + row['Catches'] != 0:
            catch_pct = row['Catches'] / (row['Drops'] + row['Catches'])
        row['Catch Pct'] = catch_pct
        return row
        
pdf = pdf.apply(catch_percent, axis=1)

In [16]:
pdf.sort_values('Goals', ascending=False)

,Drops,Catches,Catch Pct,Throwaways,Attempts,Goals
#55 Tigger Li,3.0,47.0,0.940000,10.0,60.0,11.0
#27 Daniel Zhang,5.0,155.0,0.968750,15.0,175.0,6.0
#1 Joyce Wang,2.0,55.0,0.964912,5.0,62.0,2.0
# 69 Justin Chen,2.0,8.0,0.800000,0.0,10.0,2.0
#69 Justin Chen,2.0,13.0,0.866667,3.0,18.0,1.0
Peter,0.0,2.0,1.000000,0.0,2.0,1.0
#14 Cecilia Wu,2.0,124.0,0.984127,6.0,132.0,1.0
#2 Jamie Chen,1.0,13.0,0.928571,1.0,15.0,1.0
# 36 Jason Ling,0.0,1.0,1.000000,0.0,1.0,0.0
Oscar Yao,0.0,8.0,1.000000,0.0,8.0,0.0
